In [1]:
from dotenv import load_dotenv
import os

YourAPIKey = ""

load_dotenv()

openai_api_key=os.getenv('OPENAI_API_KEY', 'YourAPIKey')

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader
llm = ChatOpenAI(model_name='gpt-3.5-turbo', openai_api_key=openai_api_key)

In [3]:
embeddings = OpenAIEmbeddings(disallowed_special=(), openai_api_key=openai_api_key)

In [4]:
root_dir = '/amos2023ws05-pipeline-config-chat-ai/src/RAG/pipelines'
docs = []

# Go through each folder
for dirpath, dirnames, filenames in os.walk(root_dir):
    
    # Go through each file
    for file in filenames:
        try: 
            # Load up the file as a doc and split
            loader = TextLoader(os.path.join(dirpath, file), encoding='utf-8')
            docs.extend(loader.load_and_split())
        except Exception as e: 
            pass

In [5]:
print (f"You have {len(docs)} documents\n")
print ("------ Start Document ------")
print (docs[0].page_content[:300])

You have 190 documents

------ Start Document ------
# Copyright 2022 RTDIP
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed 


In [6]:
docsearch = FAISS.from_documents(docs, embeddings)

In [7]:
# Get our retriever ready
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())

In [10]:
query = "I would like to use RTDIP components to read from an eventhub using 'connection string' as the connection string, and 'consumer group' as the consumer group, transform using binary to string, and edge x transformer then write to delta, generate only code without text "
output = qa.run(query)

In [11]:
print (output)

Sure! Here's an example of how you can use RTDIP components to read from an Eventhub using a connection string and consumer group, transform the data from binary to string, and then write it to a Delta table:

```python
from rtdip_sdk.pipelines.sources import SparkEventhubSource
from rtdip_sdk.pipelines.transforms import BinaryToStringTransformer, EdgeXTransformer
from rtdip_sdk.pipelines.destinations import DeltaDestination
from rtdip_sdk.pipelines.utilities import SparkSessionUtility

# Not required if using Databricks
spark = SparkSessionUtility(config={}).execute()

# Eventhub connection string and consumer group
connection_string = "YOUR_EVENTHUB_CONNECTION_STRING"
consumer_group = "YOUR_CONSUMER_GROUP"

# Create the Eventhub source
eventhub_source = SparkEventhubSource(spark=spark, options={"eventhubs.connectionString": connection_string, "eventhubs.consumerGroup": consumer_group})

# Read from Eventhub
data = eventhub_source.read_stream()

# Transform data from binary to string
